# Machine Learning Project
## Articles Analysis with Machine Learning

### Starting the project

Test...

In [2]:
import numpy as np
import pandas as pd

import os
import glob
import xml.etree.ElementTree

# Important nltk packages you might download to execute the codes
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
# nltk.download('stopwords')

%matplotlib inline
import matplotlib.pyplot as plt

from custom_entities.article import ArticleData
from custom_helpers.pandas_helper import insert_article_to_pandas

# Path of folder with all articles
path = 'C:\Users\Paulo\Documents\PyCharmProjects\papers_abstract_analysis\papers'

df = pd.DataFrame()

for filename in glob.glob(os.path.join(path, '*.xml')):
    root = xml.etree.ElementTree.parse(filename).getroot()
    article = ArticleData(root)
    df = insert_article_to_pandas(df, article)
    
df = df.fillna(0)
display(df)

,JOURNAL,across,adhd,admission,adult,affect,affective,analysis,assessment,attention,...,self-efficacy,size,sub-trajectories,symptoms,therapy,time,trajectories,treatment,urge,women
10.1016/j.psychres.2018.02.027,Psychiatry research,0,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
10.1002/pon.4678,Psycho-oncology,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,1,1,1,0,0
10.1016/j.jad.2018.02.007,Journal of affective disorders,0,1,0,1,1,0,1,1,1,...,0,0,0,1,0,0,0,0,0,0
10.1016/j.jad.2018.01.024,Journal of affective disorders,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
10.1016/j.psychres.2018.02.028,Psychiatry research,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,1,0,0,1,1


### Pubmed papers importing

Using Twitter API

### End